In [10]:
%matplotlib qt  

from matplotlib import interactive
interactive(False)

import numpy as np
import matplotlib.pyplot as plt

from data.geoms import *

from scipy.ndimage import distance_transform_edt
from scipy.interpolate import griddata, RectBivariateSpline

In [5]:
f128 = RectBivariateSpline(x128, y128, sdf_scipy128)
scipy_256_from_128 = f128(x256, y256)
plot_sdf(img256, sdf_scipy256, colorbar=True)
plot_sdf(img256, scipy_256_from_128, colorbar=True)
plt.imshow(scipy_256_from_128 - sdf_scipy256, cmap='hot')
plt.colorbar()

NameError: name 'x128' is not defined

In [ ]:
f12 = RectBivariateSpline(x256, y256, sdf_scipy256)

In [ ]:
def interpolate(x1, y1, s1, x2, y2):
    f12 = RectBivariateSpline(x1, y1, s1)
    s2_from_1 = f12(x2, y2)
    return s2_from_1

scipy_128_from_256 = interpolate(x256, y256, sdf_scipy256, x128, y128)
plot_sdf(img128, sdf_scipy128, colorbar=True)
plot_sdf(img128, scipy_128_from_256, colorbar=True)
plt.imshow(scipy_128_from_256 - sdf_scipy128, cmap='hot')
plt.colorbar()

In [ ]:
plt.imshow(sdf_scipy64 - sdf_simnet64, cmap='hot')
plt.colorbar()

In [ ]:

plt.imshow(sdf_scipy128 - sdf_simnet128, cmap='hot')
plt.colorbar()

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
plt.subplot(1,2,1)
im = plt.contourf(x256, y256, sdf_simnet256, 150, cmap='hot')
plt.colorbar()

In [139]:
def format_coord(x, y):
    col = int(x + 0.5)
    row = int(y + 0.5)
    z = X[row, col]
    return 'x=%1.4f, y=%1.4f, z=%1.4f' % (x, y, z)

In [59]:
fig, ax = plt.subplots(figsize=(20,10))
plt.subplot(1,1,1)
im = plt.contourf(x256, y256, sdf_simnet256, 150, cmap='hot')
plt.colorbar()
plt.contour(x256, y256, sdf_simnet256, levels=[-0.0001, 0.0001])
plt.show()

In [60]:
fig, ax = plt.subplots(figsize=(20,10))

plt.subplot(1,2,1)
im = plt.imshow(sdf_simnet256, cmap='hot')
plt.colorbar()
plt.contour(sdf_simnet256, levels=[-0.0001, 0.0001])

plt.subplot(1,2,2)
plt.imshow(sdf_scipy256, cmap='hot')
plt.colorbar()
plt.contour(sdf_scipy256, levels=[-0.0001, 0.0001])

ax.format_coord = format_coord
plt.show()

In [61]:
fig, ax = plt.subplots(figsize=(20,10))
plt.subplot(1,1,1)
im = plt.imshow(sdf_simnet256 - sdf_scipy256, cmap='hot')
plt.colorbar()
plt.contour(sdf_simnet256, levels=[-0.0001, 0.0001])
ax.format_coord = format_coord

plt.show()

In [183]:
def plot_kirkar(s, s2):
    fig, ax = plt.subplots(figsize=(20,10))
    plt.subplot(1,1,1)
    im = plt.imshow(s, cmap='hot')
    plt.colorbar()
    plt.clim(-0.5,1.5)
    plt.contour(s2, levels=[-0.0001, 0.0001])
    ax.format_coord = format_coord
    plt.show()
    
def get_scipy_sdf(geom, img_res):
    x, y = np.linspace(-1, 1, img_res), np.linspace(-1, 1, img_res)
    X, Y = np.meshgrid(x, y)
    sdf_simnet = geom.eval_sdf(X, Y)
    img = sdf_simnet < 0
    sdf_scipy  = -distance_transform_edt(img) + distance_transform_edt(1 - img)
    sdf_scipy /= (sdf_scipy.shape[0] // 2)
    return img, sdf_simnet, sdf_scipy

In [185]:
x1 = nGon([[0.2, 0.2], [-0.2, 0.2], [-0.5, -0.5], [0.5, -0.5]])#.scale(1.613213)
img1, sdf_simnet1, sdf_scipy1 = get_scipy_sdf(x1, 257)
plot_kirkar(sdf_scipy1, sdf_simnet1)

In [186]:
x2 = Circle(0.5).translate((0.3,0.2))
img2, sdf_simnet2, sdf_scipy2 = get_scipy_sdf(x2, 257)
plot_kirkar(sdf_scipy2, sdf_simnet2)

In [172]:
sdf_simnet2.shape

(257, 257)

In [187]:
sdf_scipy3, sdf_simnet3 = np.minimum(sdf_scipy1, sdf_scipy2), np.minimum(sdf_simnet1, sdf_simnet2)

In [188]:
plot_kirkar(sdf_scipy3, sdf_simnet3)

In [175]:
plot_kirkar(sdf_simnet3, sdf_simnet3)

In [189]:
x3 = x1.union(x3)
img4, sdf_simnet4, sdf_scipy4 = get_scipy_sdf(x3, 257)
plot_kirkar(sdf_scipy4, sdf_scipy4)

In [194]:
sdf_simnet5 = np.minimum(sdf_simnet1, sdf_simnet2)
in_in = np.logical_and(sdf_simnet1<0, sdf_simnet2<0)
sdf_simnet5[in_in] = np.minimum(-sdf_simnet1, -sdf_simnet2)[in_in]

In [195]:
plot_kirkar(sdf_simnet5, sdf_simnet5)

In [191]:
plot_kirkar(-sdf_simnet2, -sdf_simnet2)

In [196]:
img128

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [18]:
import cv2

from scipy.spatial.distance import cdist

In [9]:
def compute_edges_img(img):
    edge_lower_tresh = 50
    edge_upper_tresh = 200
    running_img = img.copy()
    running_img *= 255
    running_img = running_img.astype(np.uint8)
    edges = cv2.Canny(running_img, edge_lower_tresh, edge_upper_tresh)
    return edges

In [14]:
x, y = np.meshgrid(np.linspace(-1, 1, 128), np.linspace(-1, 1, 128))
sdf = Diamond((0.5, 0.2)).rotate(0.3).eval_sdf(x, y)
img = sdf < 0
plot_sdf(img, sdf)

In [16]:
edges = compute_edges_img(img.astype(int))

In [27]:
edges_pixels = np.array(np.where(edges == 255)).T
n_edges_pixels = edges_pixels.shape[0]

In [59]:
from scipy.spatial import ConvexHull

In [61]:
edges_pixels[:, 0], edges_pixels[:, 1]

(array([51, 51, 51, 51, 51, 51, 51, 52, 52, 52, 52, 52, 52, 52, 52, 52, 52,
        52, 52, 52, 52, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53, 53,
        53, 53, 54, 54, 54, 55, 55, 55, 56, 56, 56, 56, 57, 57, 57, 57, 58,
        58, 58, 58, 59, 59, 59, 59, 60, 60, 60, 60, 61, 61, 61, 61, 62, 62,
        62, 62, 63, 63, 63, 63, 64, 64, 64, 64, 65, 65, 65, 65, 66, 66, 66,
        66, 67, 67, 67, 67, 68, 68, 68, 68, 69, 69, 69, 69, 70, 70, 70, 70,
        71, 71, 71, 71, 72, 72, 72, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73,
        73, 73, 73, 73, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74, 74,
        74, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75, 75], dtype=int64),
 array([61, 62, 63, 64, 65, 66, 67, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58,
        59, 60, 68, 69, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48,
        69, 70, 35, 70, 71, 35, 71, 72, 35, 36, 72, 73, 36, 37, 74, 75, 37,
        38, 75, 76, 39, 40, 76, 77, 40, 41, 77, 78, 41, 42, 79, 80, 42, 43,
        80, 81, 4

In [67]:
ch = ConvexHull(edges_pixels)
plt.plot(edges_pixels[:,0], edges_pixels[:,1], 'o')
# for simplex in ch.simplices:
#     plt.plot(edges_pixels[simplex, 0], edges_pixels[simplex, 1], 'k-')
plt.show()

In [65]:
ch.simplices

array([[122, 105],
       [ 21,   0],
       [ 21,  36],
       [ 50, 137],
       [  6,   0],
       [  6,  20],
       [ 97, 105],
       [ 97,  20],
       [147, 137],
       [147, 122],
       [ 42,  36],
       [ 42,  50]], dtype=int32)

In [35]:
dist_mat = cdist(edges_pixels, edges_pixels)

In [51]:
closest_pixels_idx = np.argsort(dist_mat, axis=1)[:, 1:10]

In [53]:
dd = edges_pixels[closest_pixels_idx, :]

In [54]:
dd.shape

(148, 9, 2)

In [58]:
m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
m

<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]
<ipython-input-58-7934abdabb29>:1: RankWarning: Polyfit may be poorly conditioned
  m = [np.polyfit(ddx[:, 0], ddx[:, 1], 1)[0] for ddx in dd]

[-5.500000000000094,
 -5.3333333333333215,
 -1.8333333333333253,
 1.6666666666666834,
 3.13157894736842,
 2.8333333333333384,
 2.099999999999994,
 -5.4999999999999165,
 -5.333333333333285,
 -5.214285714285597,
 -5.124999999999907,
 0.5149572649572651,
 0.5245726495726497,
 0.5341880341880343,
 0.5331196581196583,
 -5.124999999999776,
 -5.2142857142857135,
 -5.3333333333333455,
 -5.500000000000085,
 1.7131147540983613,
 1.3703703703703656,
 -0.7967032967032994,
 -0.5989010989011015,
 -1.4333333333333387,
 -1.4189189189189226,
 -1.333333333333336,
 0.3920335429769393,
 0.4014675052410903,
 0.41090146750524126,
 0.4098532494758911,
 -5.125000000000077,
 -5.214285714285662,
 -5.333333333333434,
 -5.500000000000005,
 1.2,
 1.014999999999999,
 -0.38695652173913,
 1.0000000000000002,
 1.0000000000000002,
 -0.05281690140845115,
 1.0150000000000008,
 1.2000000000000044,
 0.24999999999999864,
 0.23958333333333248,
 1.1550000000000002,
 1.2999999999999976,
 0.5739436619718317,
 0.9553571428571447

In [50]:
plt.contourf(img)
for i in range(n_edges_pixels):
    x_1 = edges_pixels[closest_pixels_idx[i, 0]]
    x   = edges_pixels[i, :]
    x_2 = edges_pixels[closest_pixels_idx[i, 1]]
    costh = 
    th1 = np.arctan(abs(x_1[0] - x[0])/abs(x_1[1] - x[1]))
    th2 = np.arctan(abs(x_2[0] - x[0])/abs(x_2[1] - x[1]))
    print(th1, th2)
#    plt.plot([x_1[1], x[1], x_2[1]], [x_1[0], x[0], x_2[0]])
#plt.show()

0.0 0.7853981633974483
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.7853981633974483
0.0 0.7853981633974483
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.7853981633974483
0.0 0.7853981633974483
0.0 1.5707963267948966
0.0 0.7853981633974483
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.0
0.0 0.7853981633974483
1.5707963267948966 0.0
0.0 1.5707963267948966
1.5707963267948966 0.7853981633974483
1.5707963267948966 0.0
1.5707963267948966 0.0
1.5707963267948966 1.5707963267948966
0.0 1.5707963267948966
0.0 1.5707963267948966
1.5707963267948966 0.0
0.0 1.5707963267948966
1.5707963267948966 0.0
0.0 0.7853981633974483
0.0 1.5707963267948966
1.5707963267948966 0.0
0.0 0.7853981633974483
0.0 1.5707963267948966
0.0 1.5707963267948966
0.0 0.7853981633974483
0.0 1.5707963267948966
1.5707963267948966 0.0
0.0 0.7853981633974483
1.5707963267948966 0.0
1.5707963267948966 0.0
1.5707963267948966 0.0
1.5707963267948966 0.0
1.5

<ipython-input-50-1bfe57f2cfbf>:7: RuntimeWarning: divide by zero encountered in longlong_scalars
  th2 = np.arctan(abs(x_2[0] - x[0])/abs(x_2[1] - x[1]))
<ipython-input-50-1bfe57f2cfbf>:6: RuntimeWarning: divide by zero encountered in longlong_scalars
  th1 = np.arctan(abs(x_1[0] - x[0])/abs(x_1[1] - x[1]))
